In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint
import json 
import itertools
from datetime import date

## Explore data and structure of the website (Header structure)

In [2]:
url = 'https://www.realestate.com.kh/buy'
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}

category_url = []

response = requests.get(url, headers=headers)
data = bs(response.content, 'html.parser')


## Get locations list from the website

In [3]:
realestate_location = []
get_lists_of_locations = data.find_all(class_='e1ope10j0 css-ib3p51')
get_lists_of_locations

for location in get_lists_of_locations:
    realestate_location.append(location.get_text())
realestate_location

['Phnom Penh',
 'Siem Reap',
 'Sihanoukville',
 'Kep',
 'Kampot',
 'Battambang',
 'Kandal',
 'Kampong Thom',
 'Kampong Chhnang',
 'Kampong Speu',
 'Kampong Cham',
 'Koh Kong',
 'Pursat',
 'Oddar Meanchey',
 'Banteay Meanchey',
 'Pailin',
 'Kratie',
 'Mondulkiri',
 'Preah Vihear',
 'Ratanakiri',
 'Tboung Khmum',
 'Svay Rieng',
 'Takeo',
 'Stung Treng',
 'Prey Veng']

## Explore on pages listing item

In [4]:
# base_url: https://www.realestate.com.kh/buy/?page=2
# items-container
get_container_element = data.find_all(class_='item css-16n7mg5 eq4or9x0')

In [5]:
# address
addresses = []
for container_info in get_container_element:
    addresses.append(container_info.find_all(class_='address'))
addresses

[[<div class="address"><span class="location-pin"></span>Chey Chumneah, Daun Penh, Phnom Penh</div>],
 [<div class="address"><span class="location-pin"></span>Preaek Lieb, Chroy Changvar, Phnom Penh</div>],
 [<div class="address"><span class="location-pin"></span> Sangkat Buon, Sihanoukville, Sihanoukville</div>],
 [<div class="address"><span class="location-pin"></span> Sangkat Bei, Sihanoukville, Sihanoukville</div>],
 [<div class="address"><span class="location-pin"></span> Sangkat Bei, Sihanoukville, Sihanoukville</div>],
 [<div class="address"><span class="location-pin"></span> <!-- -->BKK 1, Chamkarmon, Phnom Penh</div>],
 [<div class="address"><span class="location-pin"></span> <!-- -->Svay Dankum, Siem Reap, Siem Reap</div>],
 [<div class="address"><span class="location-pin"></span> Boeung Tumpun 2, Meanchey, Phnom Penh</div>],
 [<div class="address"><span class="location-pin"></span><span class="project-link" role="button" tabindex="0">Platinum Coast</span></div>],
 [

In [6]:
# get name
names = []
for container_info in get_container_element:
    names.append(container_info.find_all(class_='featured css-1omkpog e1jqslr40'))

In [7]:
# get price value
print('\n ========================================= ========================================= \n')    
for container_info in get_container_element:
    pprint(container_info.find_all(class_='price'))


 ========================================= ========================================= 

[<div class="price"><span class="value" style="text-decoration:">$52,000</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price"><span class="value" style="text-decoration:">$290,000</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price"><span class="value" style="text-decoration:">$45,000</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price"><span class="value" style="text-decoration:">$7,500,000</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price"><span class="value" style="text-decoration:">$1,500/m²</span><span><div> </div><div class="percent listing"> </div></span></div>]
[<div class="price"><span class="value" style="text-decoration:">$85,496</span><span><div> </div><div class="percent"> </div></span></div>,
 <div class="price"><div c

In [8]:
# get info bedroom, bathroom
print('\n ========================================= ========================================= \n')    
for container_info in get_container_element:
    pprint(container_info.find_all(class_='private'))


 ========================================= ========================================= 

[<div class="private"><div class="feature"><span class="icon"></span> <span class="label">Bedrooms: 1</span><span class="short-label">1</span></div><div class="feature"><span class="icon"></span> <span class="label">Bathrooms: 1</span><span class="short-label">1</span></div><div class="feature"><span class="icon"></span> <span class="label">Floor area: 35m²</span><span class="short-label">35m²</span></div><div class="feature"><span class="icon"></span> <span class="label">Floor level: 3</span><span class="short-label">3</span></div></div>]
[<div class="private"><div class="feature"><span class="icon"></span> <span class="label">Bedrooms: 4</span><span class="short-label">4</span></div><div class="feature"><span class="icon"></span> <span class="label">Bathrooms: 5</span><span class="short-label">5</span></div></div>]
[<div class="private"><div class="feature"><span class="icon"></span> <span 

In [9]:
# get type
type_list = []
print('\n ========================================= ========================================= \n')    
for container_info in get_container_element:
    # pprint(container_info.find(class_='heading'))
    type_list.append(container_info.find(class_='heading'))
    if container_info.find("div", {"class":"heading"}):
        print(container_info.find("div", {"class":"heading"}).get_text().replace("$"," ").replace(" "," ").split(" ")[0])
    
# type(type_list)


 ========================================= ========================================= 

Apartment
Twin
Apartment
Land
Land
Project
Condo
Apartment
Condo
Project
Condo
Project
Project
Project
Borey
Condo
Project
Condo


## Function to extract the information

In [10]:
test = []
for x in range(len(addresses)):
    # Regexr area to filter data    
    bedroom_regexr = re.compile('Bedrooms: (\d+)')
    bedrooms_regexr = re.compile('Bedrooms: (\d+) to (\d+)')
    
    bathroom_regexr = re.compile('Bathrooms: (\d+)')
    bathrooms_regexr = re.compile('Bathrooms: (\d+) to (\d+)')

    floor_regexr = re.compile('Floor area: (\d+)')
    floors_regexr = re.compile('Floor area: (\d+) to (\d+)')

    
    land_regexr = re.compile('Land area: (\d+)')
    lands_regexr = re.compile('Land area: (\d+) t0 (\d+)')
    #============== End here ============================
    
    # Variable area
    realestate_type = get_container_element[x].find("div", {"class":"heading"})
    address = get_container_element[x].find(class_='address').get_text().replace('\ue92f','')
    
    price = get_container_element[x].find(class_='price').get_text().replace('\ue92f','').replace(' ','')
    
    feature = get_container_element[x].find_all(class_='feature')
    
    bedroom = bedroom_regexr.findall(str(feature))[0] if bedroom_regexr.findall(str(feature)) else None
    
    bedrooms = bedrooms_regexr.findall(str(feature))[0] if bedrooms_regexr.findall(str(feature)) else None

    
    bathroom = bathroom_regexr.findall(str(feature))[0] if bathroom_regexr.findall(str(feature)) else None
    
    bathrooms = bathrooms_regexr.findall(str(feature))[0] if bathrooms_regexr.findall(str(feature)) else None

        
    floor = floor_regexr.findall(str(feature))[0] if floor_regexr.findall(str(feature)) else None
    floors = floors_regexr.findall(str(feature))[0] if floors_regexr.findall(str(feature)) else None

    land = land_regexr.findall(str(feature))[0] if land_regexr.findall(str(feature)) else None
    lands = lands_regexr.findall(str(feature))[0] if lands_regexr.findall(str(feature)) else None
    
    realestate_type = realestate_type.get_text().replace("$"," ").replace(" "," ").split(" ")[0] if realestate_type else None
    #============== End here ============================

    # conditional before push in list
    final_bedroom = bedrooms if bedrooms else bedroom
    final_bathroom = bathrooms if bathrooms else bathroom
    final_floor = floors if floors else floor
    final_land = lands if lands else land
    #============== End here ============================
    test.append({'location': address, 'price': price, 'bedrooms': final_bedroom,
                 'bathrooms': final_bathroom, 'floors': final_floor, 'lands': final_land,
                 'type': realestate_type
                })
test



[{'location': 'Chey Chumneah, Daun Penh, Phnom Penh',
  'price': '$52,000',
  'bedrooms': '1',
  'bathrooms': '1',
  'floors': '35',
  'lands': None,
  'type': 'Apartment'},
 {'location': 'Preaek Lieb, Chroy Changvar, Phnom Penh',
  'price': '$290,000',
  'bedrooms': '4',
  'bathrooms': '5',
  'floors': None,
  'lands': None,
  'type': 'Twin'},
 {'location': ' Sangkat Buon, Sihanoukville, Sihanoukville',
  'price': '$45,000',
  'bedrooms': '1',
  'bathrooms': '1',
  'floors': '35',
  'lands': None,
  'type': 'Apartment'},
 {'location': ' Sangkat Bei, Sihanoukville, Sihanoukville',
  'price': '$7,500,000',
  'bedrooms': None,
  'bathrooms': None,
  'floors': None,
  'lands': '1025',
  'type': 'Land'},
 {'location': ' Sangkat Bei, Sihanoukville, Sihanoukville',
  'price': '$1,500/m²',
  'bedrooms': None,
  'bathrooms': None,
  'floors': None,
  'lands': '1593',
  'type': 'Land'},
 {'location': ' BKK 1, Chamkarmon, Phnom Penh',
  'price': '$85,496',
  'bedrooms': ('1', '6'),
  'bathrooms'

## Convert the loop to function

In [11]:
realestate_data_collection  = []

def extra_name_location_price_info(url, headers, sell_or_rent):
    response = requests.get(url, headers=headers)
    data = bs(response.content, 'html.parser')
    get_container_element = data.find_all(class_='item css-16n7mg5 eq4or9x0')
    for x in range(len(addresses)):
        # Regexr area to filter data    
        bedroom_regexr = re.compile('Bedrooms: (\d+)')
        bedrooms_regexr = re.compile('Bedrooms: (\d+) to (\d+)')

        bathroom_regexr = re.compile('Bathrooms: (\d+)')
        bathrooms_regexr = re.compile('Bathrooms: (\d+) to (\d+)')

        floor_regexr = re.compile('Floor area: (\d+)')
        floors_regexr = re.compile('Floor area: (\d+) to (\d+)')


        land_regexr = re.compile('Land area: (\d+)')
        lands_regexr = re.compile('Land area: (\d+) t0 (\d+)')
        #============== End here ============================

        # Variable area
        realestate_type = get_container_element[x].find("div", {"class":"heading"})
        address = get_container_element[x].find(class_='address').get_text().replace('\ue92f','')

        price = get_container_element[x].find(class_='price').get_text().replace('\ue92f','').replace(' ','')

        feature = get_container_element[x].find_all(class_='feature')

        bedroom = bedroom_regexr.findall(str(feature))[0] if bedroom_regexr.findall(str(feature)) else None

        bedrooms = bedrooms_regexr.findall(str(feature))[0] if bedrooms_regexr.findall(str(feature)) else None


        bathroom = bathroom_regexr.findall(str(feature))[0] if bathroom_regexr.findall(str(feature)) else None

        bathrooms = bathrooms_regexr.findall(str(feature))[0] if bathrooms_regexr.findall(str(feature)) else None


        floor = floor_regexr.findall(str(feature))[0] if floor_regexr.findall(str(feature)) else None
        floors = floors_regexr.findall(str(feature))[0] if floors_regexr.findall(str(feature)) else None

        land = land_regexr.findall(str(feature))[0] if land_regexr.findall(str(feature)) else None
        lands = lands_regexr.findall(str(feature))[0] if lands_regexr.findall(str(feature)) else None

        realestate_type = realestate_type.get_text().replace("$"," ").replace(" "," ").split(" ")[0] if realestate_type else None
        #============== End here ============================

        # conditional before push in list
        final_bedroom = bedrooms if bedrooms else bedroom
        final_bathroom = bathrooms if bathrooms else bathroom
        final_floor = floors if floors else floor
        final_land = lands if lands else land
        #============== End here ============================
        realestate_data_collection.append({'location': address, 'price': price, 'bedrooms': final_bedroom,
                     'bathrooms': final_bathroom, 'floors': final_floor, 'lands': final_land,
                     'realestate_category': realestate_type,'type': sell_or_rent
                    })

## Resuse the function

### Scrape the buy data set.

In [12]:
url = 'https://www.realestate.com.kh/buy'
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}

# https://www.realestate.com.kh/buy/?page=2

for page in range (50):
    extra_name_location_price_info('https://www.realestate.com.kh/buy/?page='+str(page+1), headers,'buy')
    if page == 50:
        print('done')


In [13]:
realestate_data_collection

[{'location': 'Chey Chumneah, Daun Penh, Phnom Penh',
  'price': '$52,000',
  'bedrooms': '1',
  'bathrooms': '1',
  'floors': '35',
  'lands': None,
  'realestate_category': 'Apartment',
  'type': 'buy'},
 {'location': 'Preaek Lieb, Chroy Changvar, Phnom Penh',
  'price': '$290,000',
  'bedrooms': '4',
  'bathrooms': '5',
  'floors': None,
  'lands': None,
  'realestate_category': 'Twin',
  'type': 'buy'},
 {'location': ' Sangkat Buon, Sihanoukville, Sihanoukville',
  'price': '$45,000',
  'bedrooms': '1',
  'bathrooms': '1',
  'floors': '35',
  'lands': None,
  'realestate_category': 'Apartment',
  'type': 'buy'},
 {'location': ' Sangkat Bei, Sihanoukville, Sihanoukville',
  'price': '$7,500,000',
  'bedrooms': None,
  'bathrooms': None,
  'floors': None,
  'lands': '1025',
  'realestate_category': 'Land',
  'type': 'buy'},
 {'location': ' Sangkat Bei, Sihanoukville, Sihanoukville',
  'price': '$1,500/m²',
  'bedrooms': None,
  'bathrooms': None,
  'floors': None,
  'lands': '1593',


## Scrape the loan data

In [19]:
url = 'https://www.realestate.com.kh/rent'
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}

# https://www.realestate.com.kh/buy/?page=2

for page in range (50):
    extra_name_location_price_info('https://www.realestate.com.kh/rent/?page='+str(page+1), headers, 'rent')
    if page == 50:
        print('done')


## Save and load data

In [26]:
len(realestate_data_collection)


2000

In [27]:
def save_data(title,data):
    with open(title, "w",encoding='utf-8') as outfile:
        json.dump(data, outfile, ensure_ascii=False, indent=3)

In [28]:
def load_data(title):
    with open(title,encoding='utf-8') as f:
        return json.load(f)

In [29]:
# saving data to json formate
realestate_kh = "./data/realestate_kh_50_pages_"+str(date.today())+".json"
save_data(realestate_kh,realestate_data_collection)
realestate_kh_list = load_data(realestate_kh)

## Save it CSV

In [30]:
df = pd.read_json(realestate_kh)
df.to_csv('./data/realestate_kh_50_pages_'+str(date.today())+'.csv')